In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


NEGATİF ÖRNEKLİ VERİ SETİ EĞİTİM İÇİN

In [ ]:
import json
import numpy as np
import pandas as pd
import random

# **📌 Sabitler**
NUM_FRAMES = 512  # Modelin giriş boyutu
FEATURE_DIM = 512  # CLIP özellik boyutu
NEG_SAMPLE_RATIO = 1  # Negatif örnek oranı

# **📌 CSV Dosyasını Okuma**
csv_path = "/content/drive/MyDrive/vid_sum_act/query_feature_training_all_sum.csv"
df = pd.read_csv(csv_path)

# **📌 Query Feature'larını Al (x_train_text)**
x_train_text = np.array([
    np.array(json.loads(text_feature)) for text_feature in df["text_feature"]
], dtype=np.float32)

# **📌 Segmentleri Ground Truth (y_train) Olarak Al ve One-Hot Encode Yap**
y_train = []
for segment in df["segment"]:
    try:
        # **Segment Listesini JSON'dan Parse Et**
        segment_list = json.loads(segment) if isinstance(segment, str) else segment

        # **One-Hot Encoded Maskeye Çevir (512 Uzunlukta)**
        mask = np.zeros(NUM_FRAMES, dtype=np.float32)
        for idx in segment_list:
            if 0 <= idx < NUM_FRAMES:
                mask[idx] = 1

        y_train.append(mask)

    except Exception as e:
        print(f"[WARNING] Segment işlenemedi: {segment}, Hata: {e}")

y_train = np.array(y_train, dtype=np.float32)  # (num_samples, 512)

# **📌 CLIP Feature JSON Dosya Yolları**
clip_feature_paths = [
    "/content/drive/MyDrive/vid_sum_act/clip_features_1.json",
    "/content/drive/MyDrive/vid_sum_act/clip_features_2.json",
    "/content/drive/MyDrive/vid_sum_act/clip_features_3.json",
    "/content/drive/MyDrive/vid_sum_act/clip_features_4.json",
    "/content/drive/MyDrive/vid_sum_act/clip_features_5.json",
    "/content/drive/MyDrive/vid_sum_act/clip_features_7.json"
]

# **📌 Video Feature'larını Bellekte Tutmadan Yükleme**
video_features = {}
for path in clip_feature_paths:
    with open(path, "r", encoding="utf-8") as f:
        video_features.update(json.load(f))

# **📌 Video Feature'larını Al (Giriş: x_train_video)**
x_train_video = []
for video_id in df["video_id"]:
    if video_id in video_features:
        x_train_video.append(np.array(video_features[video_id]["frame_features"], dtype=np.float32))
    else:
        continue
# **📌 RAM Kullanımını Optimize Etmek İçin Video Feature'ları Bellekten Sil**
del video_features

# **📌 Negatif Örnekleri Oluşturma**
num_neg_samples = int(len(df) * NEG_SAMPLE_RATIO)  # %50 negatif örnek

# **Mevcut query ve video ID'leri**
query_indices = list(range(len(x_train_text)))

neg_x_train_text = []
neg_x_train_video = []
neg_y_train = []

for _ in range(num_neg_samples):
    # **Farklı bir query ve yanlış bir video seç**
    neg_query_idx = random.choice(query_indices)
    neg_video_idx = random.choice(query_indices)

    # **Gerçek eşleşmelerden biri olmamasına dikkat et**
    while df.iloc[neg_query_idx]["video_id"] == df.iloc[neg_video_idx]["video_id"]:
        neg_video_idx = random.choice(query_indices)

    # **Query ve yanlış eşleşen gerçek video özelliklerini ekle**
    neg_x_train_text.append(x_train_text[neg_query_idx])  # **Yanlış query seçiliyor!**
    neg_x_train_video.append(x_train_video[neg_video_idx])  # **Yanlış video seçiliyor!**

    # **Negatif Örnek: Çıkış Sadece Sıfırlardan Oluşmalı**
    neg_y_train.append(np.zeros(NUM_FRAMES, dtype=np.float32))

# **📌 Negatif ve Pozitif Örnekleri Birleştirme**
x_train_text = np.concatenate([x_train_text, np.array(neg_x_train_text, dtype=np.float32)], axis=0)
x_train_video = np.concatenate([np.array(x_train_video, dtype=np.float32), np.array(neg_x_train_video, dtype=np.float32)], axis=0)
y_train = np.concatenate([y_train, np.array(neg_y_train, dtype=np.float32)], axis=0)

# **📌 Shuffle (Karıştırma)**
num_samples = x_train_text.shape[0]
shuffled_indices = np.random.permutation(num_samples)

x_train_text = x_train_text[shuffled_indices]
x_train_video = x_train_video[shuffled_indices]
y_train = y_train[shuffled_indices]

# **📌 Veri Seti Şekillerini Kontrol Et**
print(f"[INFO] x_train_text shape: {x_train_text.shape}")  # **(num_samples, 512)**
print(f"[INFO] x_train_video shape: {x_train_video.shape}")  # **(num_samples, 512, 512)**
print(f"[INFO] y_train shape: {y_train.shape}")  # **(num_samples, 512)**")


NEGATİF ÖRNEKLİ VERİ SETİ DOĞRULAMA VE TEST İÇİN

In [ ]:
import json
import numpy as np
import pandas as pd
import random

# **📌 Sabitler**
NUM_FRAMES = 512  # Modelin giriş boyutu
FEATURE_DIM = 512  # CLIP özellik boyutu
TEST_RATIO = 0.33  # **Test set oranı**
NEG_SAMPLE_RATIO = 1  # **Negatif örnek oranı**

# **📌 Validation İçin Kullanılacak CSV Dosyası**
val_csv_path = "/content/drive/MyDrive/vid_sum_act/query_feature_val_all_sum.csv"
df_val = pd.read_csv(val_csv_path)

# **📌 CLIP Feature JSON Dosya Yolları**
clip_feature_paths = [
    "/content/drive/MyDrive/vid_sum_act/clip_features_val_1.json",
    "/content/drive/MyDrive/vid_sum_act/clip_features_val_2.json"
]

# **📌 Video Feature'larını Bellekte Tutmadan Yükleme**
video_features = {}
for path in clip_feature_paths:
    with open(path, "r", encoding="utf-8") as f:
        video_features.update(json.load(f))

# **📌 Validation ve Test Setlerini Ayırma**
video_ids = df_val["video_id"].unique()
np.random.shuffle(video_ids)  # **Video ID'leri karıştır**
test_size = int(len(video_ids) * TEST_RATIO)
test_videos = set(video_ids[:test_size])  # **%33 Test**
val_videos = set(video_ids[test_size:])  # **%67 Validation**

df_test = df_val[df_val["video_id"].isin(test_videos)].reset_index(drop=True)
df_val = df_val[df_val["video_id"].isin(val_videos)].reset_index(drop=True)

def prepare_data(df):
    """Validation ve Test için x_text, x_video, y verilerini hazırlar."""

    # **📌 Query Feature'larını CLIP formatına uygun şekilde yükleme**
    x_text = np.array([
        np.array(json.loads(text_feature)) for text_feature in df["text_feature"]
    ], dtype=np.float32)

    # **📌 Video Feature'larını yükleme**
    x_video = []
    for video_id in df["video_id"]:
        if video_id in video_features:
            x_video.append(np.array(video_features[video_id]["frame_features"], dtype=np.float32))
        else:
            continue
    x_video = np.array(x_video, dtype=np.float32)  # (num_samples, 512, 512)

    # **📌 Ground Truth'ları Hazırlama ve One-Hot Encode Yapma**
    y = []
    for segment in df["segment"]:
        try:
            segment_list = json.loads(segment) if isinstance(segment, str) else segment

            # **One-Hot Encoded Maskeye Çevir (512 Uzunlukta)**
            mask = np.zeros(NUM_FRAMES, dtype=np.float32)
            for idx in segment_list:
                if 0 <= idx < NUM_FRAMES:
                    mask[idx] = 1

            y.append(mask)

        except Exception as e:
            print(f"[WARNING] Segment işlenemedi: {segment}, Hata: {e}")

    y = np.array(y, dtype=np.float32)  # (num_samples, 512)

    # **📌 Negatif Örnekler Oluşturma (Verinin %50'si Kadar)**
    num_neg_samples = int(len(df) * NEG_SAMPLE_RATIO)  # %50 negatif örnek

    query_indices = list(range(len(x_text)))

    neg_x_text = []
    neg_x_video = []
    neg_y = []

    for _ in range(num_neg_samples):
        # **Farklı bir query ve yanlış bir video seç**
        neg_query_idx = random.choice(query_indices)
        neg_video_idx = random.choice(query_indices)

        # **Gerçek eşleşmelerden biri olmamasına dikkat et**
        while df.iloc[neg_query_idx]["video_id"] == df.iloc[neg_video_idx]["video_id"]:
            neg_video_idx = random.choice(query_indices)

        # **Yanlış eşleşmiş gerçek video ve query seç**
        neg_x_text.append(x_text[neg_query_idx])  # **Yanlış query**
        neg_x_video.append(x_video[neg_video_idx])  # **Yanlış video**

        # **Negatif Örnek: Çıkış Sadece Sıfırlardan Oluşmalı**
        neg_y.append(np.zeros(NUM_FRAMES, dtype=np.float32))

    # **📌 Negatif ve Pozitif Örnekleri Birleştirme**
    x_text = np.concatenate([x_text, np.array(neg_x_text, dtype=np.float32)], axis=0)
    x_video = np.concatenate([x_video, np.array(neg_x_video, dtype=np.float32)], axis=0)
    y = np.concatenate([y, np.array(neg_y, dtype=np.float32)], axis=0)

    # **📌 Shuffle (Karıştırma)**
    num_samples = x_text.shape[0]
    shuffled_indices = np.random.permutation(num_samples)

    x_text = x_text[shuffled_indices]
    x_video = x_video[shuffled_indices]
    y = y[shuffled_indices]

    return x_text, x_video, y

# **📌 Validation ve Test Verilerini Hazırla (Negatif Örneklerle)**
x_val_text, x_val_video, y_val = prepare_data(df_val)
x_test_text, x_test_video, y_test = prepare_data(df_test)

# **📌 RAM Kullanımını Optimize Etmek İçin Temizleme**
del video_features

# **📌 Veri Seti Şekillerini Kontrol Et**
print(f"[INFO] x_val_text shape: {x_val_text.shape}")  # **(num_samples, 512)**
print(f"[INFO] x_val_video shape: {x_val_video.shape}")  # **(num_samples, 512, 512)**
print(f"[INFO] y_val shape: {y_val.shape}")  # **(num_samples, 512)**

print(f"[INFO] x_test_text shape: {x_test_text.shape}")  # **(num_samples, 512)**
print(f"[INFO] x_test_video shape: {x_test_video.shape}")  # **(num_samples, 512, 512)**
print(f"[INFO] y_test shape: {y_test.shape}")  # **(num_samples, 512)**")


In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, BatchNormalization, Dropout, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow.keras.backend as K

# **📌 Transformer Encoder Katmanı**
class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.05):
        super(TransformerEncoder, self).__init__()
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.norm1 = BatchNormalization()  # **Batch Normalization Eklendi**
        self.norm2 = BatchNormalization()
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)

    def call(self, inputs, training=False):
        attn_output = self.attention(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.norm1(inputs + attn_output, training=training)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.norm2(out1 + ffn_output, training=training)

# **📌 Cross-Attention Katmanı (Video & Query Etkileşimi)**
class CrossAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, dropout_rate=0.05):
        super(CrossAttentionLayer, self).__init__()
        self.cross_attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.norm = BatchNormalization()  # **Batch Normalization Eklendi**
        self.dropout = Dropout(dropout_rate)

    def call(self, video_features, query_features, training=False):
        query_expanded = tf.expand_dims(query_features, axis=1)  # (batch, 1, 512)
        query_tiled = tf.tile(query_expanded, [1, 512, 1])  # (batch, 512, 512)

        attn_output = self.cross_attention(query_tiled, video_features)
        attn_output = self.dropout(attn_output, training=training)
        return self.norm(video_features + attn_output, training=training)

# **📌 Model Girdileri**
video_input = Input(shape=(512, 512), name="video_features")  # (512, 512) video CLIP özellikleri

# **📌 Video Özelliklerini İşle (Transformer Encoder)**
video_encoded = TransformerEncoder(embed_dim=512, num_heads=8, ff_dim=1024)(video_input)

# **📌 Query Özellikleri: Modelin Girişinden Alınıyor**
query_input = Input(shape=(512,), name="query_embedding")  # (512,) query CLIP özelliği

# **📌 Query Encoder**
query_encoded = query_input

# **📌 Video & Query Fusion (Cross-Attention)**
cross_attention_output = CrossAttentionLayer(embed_dim=512, num_heads=8)(video_encoded, query_encoded)

# **📌 Transformer Encoder ile Özeti İşle**
decoder_output = TransformerEncoder(embed_dim=512, num_heads=8, ff_dim=1024)(cross_attention_output)

# **📌 Çıktıyı Frame Seçimi Olarak Al (Binary Classification)**
output = Dense(1, activation="sigmoid")(decoder_output)  # (512, 1) → frame selection

# **📌 Modeli Tanımla**
model = Model(inputs=[video_input, query_input], outputs=output, name="QueryBasedVideoSummarization")

# **📌 Hybrid Loss (BCE + Dice Loss)**
def dice_loss(y_true, y_pred):
    smooth = 1.0
    y_pred = K.clip(y_pred, 1e-7, 1.0 - 1e-7)  # **Normalize ettik**
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def hybrid_loss(y_true, y_pred):
    """
    Binary Crossentropy ve Dice Loss kombinasyonu.
    """
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return 0.3 * bce + 0.7 * dice  # **Dice Loss'un ağırlığını artırdık**

# **📌 Modeli Derle**
optimizer = Adam(learning_rate=2e-5)  # **Daha stabil öğrenme için düşük öğrenme oranı**
model.compile(optimizer=optimizer, loss=hybrid_loss, metrics=["accuracy"])

# **📌 Modeli Kaydetmek İçin Klasör Belirleme**
save_dir = "/content/drive/MyDrive/vid_sum_act/"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# **📌 Checkpoint (Her Epoch Sonrası En İyi Modeli Kaydet)**
checkpoint_path = os.path.join(save_dir, "query_based_video_summarization.keras")
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor="val_loss",  # En iyi validation loss'a göre kaydet
    save_best_only=True,  # Sadece en iyi modeli kaydet
    save_weights_only=False,
    verbose=1
)

# **📌 Early Stopping (Overfitting'i Önlemek İçin)**
early_stopping_callback = EarlyStopping(
    monitor="val_loss",
    patience=10,  # Eğer 10 epoch boyunca iyileşme olmazsa dur
    restore_best_weights=True,  # En iyi ağırlıkları yükle
    verbose=1
)

# **📌 Eğitim Parametreleri**
batch_size = 16
epochs = 50  # Maksimum 50 epoch

# **📌 Modeli Eğit**
history = model.fit(
    [x_train_video, x_train_text],  # Modelin girişleri
    y_train,  # Ground truth (One-hot encoded mask)
    validation_data=([x_val_video, x_val_text], y_val),  # Validation set
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    callbacks=[early_stopping_callback, checkpoint_callback]
)

# **📌 En İyi Modeli Kaydet**
model.save(os.path.join(save_dir, "final_query_based_video_summarization.keras"))
print(f"📌 Model başarıyla kaydedildi: {save_dir}")


In [ ]:
# Modelin ağırlıklarını kaydetme

model.save_weights('/content/drive/MyDrive/vid_sum_act/queryselfattn_weights.weights.h5')
#model.load_weights("/content/drive/MyDrive/vid_sum_act/b-query_decodinp_based_video_summarization.weights.h5")